
# Build baseline tfrs model 

Look inside of `two_tower_src/` for the source code and model code

This notebook constructs the two tower model and saves the model to GCS

We will use managed Tensorboard for training. Before beginning, create a new tensorboard instance by going to Vertex -> Experiments -> Tensorboard Instances -> Create

![](img/create-a-tb.png)

In [1]:
# !pip install tensorflow-recommenders --user

#### Restart kernel after installation

In [1]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 
path = 'gs://jt-tfrs-output-v2' #TODO change to your model directory

In [2]:
import os

os.environ['TF_GPU_THREAD_MODE']='gpu_private'
os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'

In [3]:
import json
import pickle as pkl

import tensorflow as tf
import logging
import time

import tensorflow_recommenders as tfrs

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


from google.cloud import storage

# from two_tower_src import two_tower as tt
from two_tower_jt import two_tower as tt
#inside this tt module the data parsing functions, candidate dataset and model classes are found

2022-12-06 16:07:13.034908: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-06 16:07:13.666448: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:214] Using CUDA malloc Async allocator for GPU: 0
2022-12-06 16:07:13.666712: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38238 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


## Create Dataset for local training and testing

Inspect the contents of the directory - you can change parameters in the header of the `two_tower.py` script

In [4]:
!tree two_tower_jt

two_tower_jt
├── __init__.py
├── __pycache__
│   ├── __init__.cpython-37.pyc
│   └── two_tower.cpython-37.pyc
├── task.py
└── two_tower.py

1 directory, 5 files


## Playlist dataset

In [4]:
from google.cloud import aiplatform as vertex_ai

batch_size = 1024*16

client = storage.Client()
# options = tf.data.Options()
# options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.AUTO

def full_parse(data):
    # used for interleave - takes tensors and returns a tf.dataset
    data = tf.data.TFRecordDataset(data)
    return data

### train files

In [5]:

train_dir = 'spotify-data-regimes'
train_dir_prefix = 'jtv5/train_last_5_feats_v4/'

train_files = []
for blob in client.list_blobs(f'{train_dir}', prefix=f'{train_dir_prefix}', delimiter="/"):
    train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))
    
# train_dataset = tf.data.Dataset.from_tensor_slices(train_files).prefetch(
#     tf.data.AUTOTUNE,
# )

# train_dataset = train_dataset.interleave(
#     full_parse,
#     cycle_length=tf.data.AUTOTUNE, 
#     num_parallel_calls=tf.data.AUTOTUNE,
#     deterministic=False,
# ).shuffle(batch_size*4, reshuffle_each_iteration=False).map(tt.parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE,).batch(
#     batch_size 
# ).prefetch(
#     tf.data.AUTOTUNE,
# ).with_options(options)

# train_files = [
#     'gs://spotify-data-regimes/jtv5/train_last_5_feats_v4/-00000-of-00899.tfrecords',
#     'gs://spotify-data-regimes/jtv5/train_last_5_feats_v4/-00001-of-00899.tfrecords',
# ]

train_dataset = tf.data.TFRecordDataset(train_files)
train_parsed = train_dataset.map(tt.parse_tfrecord)


In [6]:
train_parsed

<MapDataset element_spec={'album_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'album_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_followers_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_genres_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_pop_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'avg_art_followers_pl_new': TensorSpec(shape=(), dtype=tf.float32, name=None), 'avg_artist_pop_pl_new': TensorSpec(shape=(), dtype=tf.float32, name=None), 'avg_track_pop_pl_new': TensorSpec(shape=(), dtype=tf.float32, name=None), 'duration_ms_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'num_pl_albums_new': TensorSpec(shape=(), dtype=tf.float32, name=None), 'num_pl_artists_new': TensorSpec(shape=(), dtype=tf.float32, name=None), 'num_pl_followers_src': TensorSpec(shape=(

In [8]:
from pprint import pprint

for features in train_parsed.skip(8).take(4):
    pprint(features)
    print("_______________")


{'album_name_can': <tf.Tensor: shape=(), dtype=string, numpy=b'Al Principio'>,
 'album_name_pl': <tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'Amor, Familia Y Respeto', b'Controlmania', b'Al Principio',
       b'40 \xc3\x89xitos', b'Grandes Exitos', b'Desde La Cantina'],
      dtype=object)>,
 'album_uri_can': <tf.Tensor: shape=(), dtype=string, numpy=b'spotify:album:56cikzve6YOgyurLWMCcVP'>,
 'album_uri_pl': <tf.Tensor: shape=(6,), dtype=string, numpy=
array([b'spotify:album:5XuNzqgx79H4Z2jhfdzFFQ',
       b'spotify:album:2kwllRxwlNWED6dWA7jGqb',
       b'spotify:album:56cikzve6YOgyurLWMCcVP',
       b'spotify:album:7L9TeAqSwkvSkFzyogoVxd',
       b'spotify:album:6jSBHoKlkbnoLzNKnvmr3R',
       b'spotify:album:5516JjJfiO6IGxBuPjZIyy'], dtype=object)>,
 'artist_followers_can': <tf.Tensor: shape=(), dtype=float32, numpy=116537.0>,
 'artist_genres_can': <tf.Tensor: shape=(), dtype=string, numpy=b"'norteno', 'tejano', 'tex-mex'">,
 'artist_genres_pl': <tf.Tensor: shape=(6,), dtype

In [10]:
for raw_record in train_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "album_name_can"
    value {
      bytes_list {
        value: "Alex Goot & Friends, Vol. 3"
      }
    }
  }
  feature {
    key: "album_name_pl"
    value {
      bytes_list {
        value: "Be Not Nobody"
        value: "Let Go"
        value: "Goodbye Lullaby"
        value: "The Best Damn Thing"
        value: "Alex Goot & Friends, Vol. 3"
        value: "Two Lanes Of Freedom"
      }
    }
  }
  feature {
    key: "album_uri_can"
    value {
      bytes_list {
        value: "spotify:album:0GhppB5IFroTmdP5HxQKE0"
      }
    }
  }
  feature {
    key: "album_uri_pl"
    value {
      bytes_list {
        value: "spotify:album:7D6BFTArx2ajtkKRVXIKO2"
        value: "spotify:album:7h6XeTzy0SRXDrFJeA9gO7"
        value: "spotify:album:3WKHuDtWB0Ota02oXE9f9S"
        value: "spotify:album:0XypvgyeJm4mNjH4QRHmYR"
        value: "spotify:album:0GhppB5IFroTmdP5HxQKE0"
        value: "spotify:album:1O3BsjGx9plSOJ036ZY4Fl"
      }
    }
  }
  feature {
  

In [9]:
for x in train_parsed.batch(4).skip(2).take(1):
    print(x['track_valence_pl'])

tf.Tensor(
[[0.935  0.763  0.737  0.888  0.83   0.665 ]
 [0.273  0.228  0.354  0.444  0.397  0.0927]
 [0.327  0.235  0.391  0.115  0.14   0.352 ]
 [0.69   0.547  0.779  0.741  0.95   0.766 ]], shape=(4, 6), dtype=float32)


2022-12-06 15:52:59.171078: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.


### Validation files

In [7]:
valid_dir = 'spotify-data-regimes'
valid_dir_prefix = 'jtv5/valid_last_5_feats_v4/'

valid_files = []
for blob in client.list_blobs(f'{valid_dir}', prefix=f'{valid_dir_prefix}', delimiter="/"):
    valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))


# valid_dataset = tf.data.Dataset.from_tensor_slices(valid_files).prefetch(
#     tf.data.AUTOTUNE,
# )

# valid_dataset = valid_dataset.interleave(
#     full_parse,
#     num_parallel_calls=tf.data.AUTOTUNE,
#     cycle_length=tf.data.AUTOTUNE, 
#     deterministic=False,
# ).map(tt.parse_tfrecord, num_parallel_calls=tf.data.AUTOTUNE).batch(
#     batch_size
# ).prefetch(
#     tf.data.AUTOTUNE,
# ).with_options(options)

valid_dataset = tf.data.TFRecordDataset(valid_files)
valid_parsed = valid_dataset.map(tt.parse_tfrecord)

# valid_dataset = valid_dataset #.cache() #1gb machine mem + 400 MB in candidate ds (src/two-tower.py)

In [8]:
valid_parsed

<MapDataset element_spec={'album_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'album_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_followers_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_genres_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_name_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'artist_pop_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'artist_uri_can': TensorSpec(shape=(), dtype=tf.string, name=None), 'avg_art_followers_pl_new': TensorSpec(shape=(), dtype=tf.float32, name=None), 'avg_artist_pop_pl_new': TensorSpec(shape=(), dtype=tf.float32, name=None), 'avg_track_pop_pl_new': TensorSpec(shape=(), dtype=tf.float32, name=None), 'duration_ms_can': TensorSpec(shape=(), dtype=tf.float32, name=None), 'num_pl_albums_new': TensorSpec(shape=(), dtype=tf.float32, name=None), 'num_pl_artists_new': TensorSpec(shape=(), dtype=tf.float32, name=None), 'num_pl_followers_src': TensorSpec(shape=(

In [9]:
for x in valid_parsed.batch(3).take(1):
    print(x['track_uri_pl'])

tf.Tensor(
[[b'spotify:track:203zTFd1zbfG0fiOG9OREv'
  b'spotify:track:3uD4aRM8QoEAAhwpOke7QU'
  b'spotify:track:6r7Pl4njxpio06Amy2fxrA'
  b'spotify:track:3tZwKujD64Ad6vlDVxZOrg'
  b'spotify:track:6KyOCzf2A2jjROH4ZokTEw'
  b'spotify:track:6AwPEpVjrWnJ0Avv5Krvgr']
 [b'spotify:track:1d15QaDMYLlzPGaYWMEw06'
  b'spotify:track:23aRQxzv8AbUOAV4czlNmp'
  b'spotify:track:7zJg7aNCvTKW9EtG1Dvzkl'
  b'spotify:track:7KVQiFRrq9UWYjgsbL8woP'
  b'spotify:track:3yvMWXSQs2W5IVNBHkgZom'
  b'spotify:track:5MCG4XcVcvCOXWpSg2cfRC']
 [b'spotify:track:1mqlc0vEP9mU1kZgTi6LIQ'
  b'spotify:track:78WVLOP9pN0G3gRLFy1rAa'
  b'spotify:track:44AyOl4qVkzS48vBsbNXaC'
  b'spotify:track:3MrRksHupTVEQ7YbA0FsZK'
  b'spotify:track:72xFgK8VpmUILQcXMNCLGN'
  b'spotify:track:1OOtq8tRnDM8kG2gqUPjAj']], shape=(3, 6), dtype=string)


2022-12-06 16:05:31.440232: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_uri_pl.  Can't parse serialized Example.


### Candidate files

In [10]:
# candidate_files = ['gs://spotify-data-regimes/jtv1-candidates/candidates-00000-of-00001.tfrecords'] # TODO: parametrize
# candidate_files = ['gs://spotify-data-regimes/jtv5/candidates/candidates-00000-of-00001.tfrecords']   # removed track_playlist_titles
candidate_files = ['gs://spotify-data-regimes/jtv5/candidates/candidates-00000-of-00001.tfrecords']

candidate_dataset = tf.data.TFRecordDataset(candidate_files)
parsed_candidate_dataset = candidate_dataset.map(tt.parse_candidate_tfrecord_fn)
# parsed_candidate_dataset

In [9]:
# for x in parsed_candidate_dataset.batch(2).take(1):
#     print(x)

# Local Training

Compile the model
Review the details of the model layers

### Adapt the text vectorizors - copy/paste to run one time

We are accessing the `TextVectorizor` layers in the model via the layer print-outs above

```python
# adpat the text vectorizors
model.query_tower.layers[3].layers[0].adapt(
    train_dataset.unbatch().batch(40000).map(lambda x: x['artist_name_pl']) #artist name pl
model.query_tower.layers[5].layers[0].adapt(
    train_dataset.unbatch().batch(40000).map(lambda x: x['track_name_pl']) #track name pl
model.query_tower.layers[7].layers[0].adapt(
    train_dataset.unbatch().batch(40000).map(lambda x: x['album_name_pl']) #album name pl
model.query_tower.layers[11].layers[0].adapt(
    train_dataset.unbatch().batch(40000).map(lambda x: x['artist_genres_pl']) #artist genres pl

model.candidate_tower.layers[1].layers[0].adapt(
    train_dataset.unbatch().batch(40000).map(lambda x: x['track_name_can'])) #track name can
model.candidate_tower.layers[2].layers[0].adapt(
    train_dataset.unbatch().batch(40000).map(lambda x: x['album_name_can'])) #album name can
model.candidate_tower.layers[10].layers[0].adapt(
    train_dataset.unbatch().batch(40000).map(lambda x: x['artist_genres_can'])) #artist genres can
```


## Save the vocab dictionary for later so you will not have to adapt

```python
vocab_dict = {
    'artist_name_pl' : model.query_tower.layers[3].layers[0].get_vocabulary(), #artist name pl
    'track_name_pl' : model.query_tower.layers[5].layers[0].get_vocabulary(), #track name pl
    'album_name_pl' : model.query_tower.layers[7].layers[0].get_vocabulary(), #album name pl
    'artist_genres_pl' : model.query_tower.layers[11].layers[0].get_vocabulary(), #artist genres pl

    'track_name_can' : model.candidate_tower.layers[1].layers[0].get_vocabulary(), #track name can
    'album_name_can' : model.candidate_tower.layers[2].layers[0].get_vocabulary(), #album name can
    'artist_genres_can' : model.candidate_tower.layers[10].layers[0].get_vocabulary(), #artist genres can
}
```


#### Save the vocabs

```python
import pickle as pkl

filehandler = open('vocab_dict.pkl', 'wb')
pkl.dump(vocab_dict, filehandler)

filehandler.close()

tt.upload_blob('two-tower-models', 'vocab_dict.pkl', 'vocabs/vocab_dict.pkl')
````

In [10]:
# # jw vocab
# os.system('gsutil cp gs://two-tower-models/vocabs/vocab_dict.pkl .')

# BUCKET_DATA_jw = 'two-tower-models'
# VOCAB_LOCAL_FILE_jw = 'vocab_dict.pkl'
# VOCAB_GCS_OBJ_jw = 'vocabs/vocab_dict.pkl'

# filehandler = open(f'{VOCAB_LOCAL_FILE_jw}', 'rb')
# vocab_dict_jw = pkl.load(filehandler)
# filehandler.close()
# vocab_dict_jw

In [ ]:
MAX_TOKENS=20000 #50000

# pl_name_src
pl_name_src_text_layer = tf.keras.layers.TextVectorization() # max_tokens=MAX_TOKENS,ngrams=2,
pl_name_src_text_layer.adapt(train_parsed.map(lambda x: x['pl_name_src'])) # lambda x, y: y[key name]

In [5]:
MAX_TOKENS=20000 #50000

# ==========================================================================================
# PLAYLIST TOWER
# ==========================================================================================
# pl_name_src
pl_name_src_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
pl_name_src_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['pl_name_src']))

# track_name_pl
track_name_pl_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
track_name_pl_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['track_name_pl']))

# artist_name_pl
artist_name_pl_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
artist_name_pl_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['artist_name_pl']))

# album_name_pl
album_name_pl_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
album_name_pl_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['album_name_pl']))

# artist_genres_pl
artist_genres_pl_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
artist_genres_pl_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['artist_genres_pl']))

# # tracks_playlist_titles_pl
# tracks_playlist_titles_pl_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
# tracks_playlist_titles_pl_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['tracks_playlist_titles_pl']))

# ==========================================================================================
# CANDIDATE TOWER
# ==========================================================================================

# track_name_can
track_name_can_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
track_name_can_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['track_name_can']))

# artist_name_can
artist_name_can_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
artist_name_can_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['artist_name_can']))

# album_name_can
album_name_can_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
album_name_can_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['album_name_can']))

# artist_genres_can
artist_genres_can_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
artist_genres_can_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['artist_genres_can']))

# # track_pl_titles_can
# track_pl_titles_can_text_layer = tf.keras.layers.TextVectorization(max_tokens=MAX_TOKENS,ngrams=2,)
# track_pl_titles_can_text_layer.adapt(train_parsed.batch(1000).map(lambda x: x['track_pl_titles_can']))


2022-12-06 07:21:51.196080: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.
2022-12-06 07:21:51.405048: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.
2022-12-06 07:21:51.412763: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.


InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  Key: track_valence_pl.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[IteratorGetNext]]
	 [[StringSplit/StringSplitV2/_2]]
  (1) INVALID_ARGUMENT:  Key: track_valence_pl.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_adapt_step_605]

2022-12-06 07:21:51.548160: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.
2022-12-06 07:21:51.749236: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.
2022-12-06 07:21:52.020817: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.
2022-12-06 07:21:52.020930: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.
2022-12-06 07:21:52.023126: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.
2022-

In [11]:
# 1st adapt
# gsutil cp gs://spotify-data-regimes/jtv1/vocabs/vocab_dict.pkl .

# BUCKET_DATA = 'spotify-data-regimes'
# VOCAB_LOCAL_FILE = 'vocab_dict.pkl'
# VOCAB_GCS_OBJ = 'jtv1/vocabs/vocab_dict.pkl'

# vocab_dict = {
#     'pl_name_src' : ['empty'],
#     'track_name_pl' : ['empty'],
#     'artist_name_pl' : ['empty'],
#     'artist_genres_pl' : ['empty'],
#     'tracks_playlist_titles_pl' : ['empty'],
#     'track_name_can' : ['empty'],
#     'artist_name_can' : ['empty'],
#     'album_name_can' : ['empty'],
#     'artist_genres_can' : ['empty'],
#     'track_pl_titles_can' : ['empty'],
# }

In [25]:
# # write vocab to gcs

# filehandler = open('vocab_dict.pkl', 'wb')

# pkl.dump(vocab_dict, filehandler)

# filehandler.close()

# tt.upload_blob(f'{BUCKET_DATA}', f'{VOCAB_LOCAL_FILE}', f'{VOCAB_GCS_OBJ}')

File vocab_dict.pkl uploaded to jtv1/vocabs/vocab_dict.pkl.


In [60]:
# filehandler = open(f'{VOCAB_LOCAL_FILE}', 'rb')
# vocab_dict_load = pkl.load(filehandler)
# filehandler.close()
# vocab_dict_load

{'pl_name_src': ['empty'], 'track_name_pl': ['empty'], 'artist_name_pl': ['empty'], 'artist_genres_pl': ['empty'], 'tracks_playlist_titles_pl': ['empty'], 'track_name_can': ['empty'], 'artist_name_can': ['empty'], 'album_name_can': ['empty'], 'artist_genres_can': ['empty'], 'track_pl_titles_can': ['empty']}

In [19]:
from two_tower_jt import two_tower as tt

layer_sizes=[512,256]

model = tt.TheTwoTowers(layer_sizes) 
# model

In [20]:
## Quick look at the layers
print("Playlist (query) Tower:")

for i, l in enumerate(model.query_tower.layers):
    print(i, l.name)

Playlist (query) Tower:
0 pl_name_src_text_embedding
1 pl_collaborative_emb_model
2 num_pl_followers_src_emb_model
3 pl_duration_ms_new_emb_model
4 num_pl_songs_new_emb_model
5 num_pl_artists_new_emb_model
6 num_pl_albums_new_emb_model
7 avg_track_pop_pl_new_emb_model
8 avg_artist_pop_pl_new_emb_model
9 avg_art_followers_pl_new_emb_model
10 track_uri_pl_emb_model
11 track_name_pl_emb_model
12 artist_uri_pl_emb_model
13 artist_name_pl_emb_model
14 album_uri_pl_emb_model
15 album_name_pl_emb_model
16 artist_genres_pl_emb_model
17 duration_ms_songs_pl_emb_model
18 track_pop_pl_emb_model
19 artist_pop_pl_emb_model
20 artists_followers_pl_emb_model
21 track_danceability_pl_emb_model
22 track_energy_pl_emb_model
23 track_key_pl_emb_model
24 track_loudness_pl_emb_model
25 track_mode_pl_emb_model
26 track_speechiness_pl_emb_model
27 track_acousticness_pl_emb_model
28 track_instrumentalness_pl_emb_model
29 track_liveness_pl_emb_model
30 track_valence_pl_emb_model
31 track_tempo_pl_emb_model
32 

In [21]:
print("Track (candidate) Tower:")
for i, l in enumerate(model.candidate_tower.layers):
    print(i, l.name)

Track (candidate) Tower:
0 track_uri_can_emb_model
1 track_name_can_emb_model
2 artist_uri_can_emb_model
3 artist_name_can_emb_model
4 album_uri_can_emb_model
5 album_name_can_emb_model
6 duration_ms_can_emb_model
7 track_pop_can_emb_model
8 artist_pop_can_emb_model
9 artist_genres_can_emb_model
10 artists_followers_can_emb_model
11 track_danceability_can_emb_model
12 track_energy_can_emb_model
13 track_key_can_emb_model
14 track_loudness_can_emb_model
15 track_mode_can_emb_model
16 track_speechiness_can_emb_model
17 track_acousticness_can_emb_model
18 track_instrumentalness_can_emb_model
19 track_liveness_can_emb_model
20 track_valence_can_emb_model
21 track_tempo_can_emb_model
22 track_time_signature_can_emb_model
23 candidate_dense_layers


In [22]:
model.query_tower.layers[11].layers[0]

In [23]:
# # adpat the text vectorizors
model.query_tower.layers[0].layers[0].adapt(
    train_parsed.batch(1000).map(lambda x: x['pl_name_src']))

# model.query_tower.layers[11].layers[0].adapt(
#     train_parsed.batch(40000).map(lambda x: x['track_name_pl']))

# model.query_tower.layers[13].layers[0].adapt(
#     train_parsed.batch(40000).map(lambda x: x['artist_name_pl']))

# model.query_tower.layers[15].layers[0].adapt(
#     train_parsed.batch(40000).map(lambda x: x['album_name_pl']))

# model.query_tower.layers[16].layers[0].adapt(
#     train_parsed.batch(40000).map(lambda x: x['artist_genres_pl']))

# # model.query_tower.layers[17].layers[0].adapt(
# #     train_parsed.batch(40000).map(lambda x: x['tracks_playlist_titles_pl']))
    
# # candidate tower layers
# model.candidate_tower.layers[1].layers[0].adapt(
#     train_parsed.batch(40000).map(lambda x: x['track_name_can']))

# model.candidate_tower.layers[3].layers[0].adapt(
#     train_parsed.batch(40000).map(lambda x: x['artist_name_can']))

# model.candidate_tower.layers[5].layers[0].adapt(
#     train_parsed.batch(40000).map(lambda x: x['album_name_can']))

# model.candidate_tower.layers[9].layers[0].adapt(
#     train_parsed.batch(40000).map(lambda x: x['artist_genres_can']))

# # model.candidate_tower.layers[11].layers[0].adapt(
# #     train_parsed.batch(40000).map(lambda x: x['track_pl_titles_can']))

2022-12-06 13:02:36.733498: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.


InvalidArgumentError: Graph execution error:

2 root error(s) found.
  (0) INVALID_ARGUMENT:  Key: track_valence_pl.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[IteratorGetNext]]
	 [[None_lookup_table_find/LookupTableFindV2/_24]]
  (1) INVALID_ARGUMENT:  Key: track_valence_pl.  Can't parse serialized Example.
	 [[{{node ParseSingleExample/ParseExample/ParseExampleV2}}]]
	 [[IteratorGetNext]]
0 successful operations.
0 derived errors ignored. [Op:__inference_adapt_step_3911]

2022-12-06 13:02:36.867952: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.
2022-12-06 13:02:36.877955: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at example_parsing_ops.cc:94 : INVALID_ARGUMENT: Key: track_valence_pl.  Can't parse serialized Example.


### Compile Model

In [ ]:
# compile model
LR = .1
opt = tf.keras.optimizers.Adagrad(LR)
model.compile(optimizer=opt)

### Local Training

Setup tensorboard below so training is visible and we can inspect the graph

In [10]:
TB_RESOURCE_NAME = 'projects/934903580331/locations/us-central1/tensorboards/7336372589079560192' #fqn - project number then tensorboard id
invoke_time = time.strftime("%Y%m%d-%H%M%S")
EXPERIMENT_NAME = f'spotify-singe-node-train-full-data-v7-01'
RUN_NAME = EXPERIMENT_NAME+'run'+time.strftime("%Y%m%d-%H%M%S")
LOG_DIR = path+"/tb-logs/"+EXPERIMENT_NAME


def get_upload_logs_to_manged_tb_command(ttl_hrs, oneshot="false"):
    """
    Run this and copy/paste the command into terminal to have 
    upload the tensorboard logs from this machine to the managed tb instance
    Note that the log dir is at the granularity of the run to help select the proper
    timestamped run in Tensorboard
    You can also run this in one-shot mode after training is done 
    to upload all tb objects at once
    """
    return(f"""tb-gcp-uploader --tensorboard_resource_name={TB_RESOURCE_NAME} \
      --logdir={LOG_DIR} \
      --experiment_name={EXPERIMENT_NAME} \
      --one_shot={oneshot} \
      --event_file_inactive_secs={60*60*ttl_hrs}""")

vertex_ai.init(experiment=EXPERIMENT_NAME)
    

# we are going to ecapsulate this one-shot log uploader via a custom callback:

class UploadTBLogsBatchEnd(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        os.system(get_upload_logs_to_manged_tb_command(ttl_hrs = 5, oneshot="true"))

In [11]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir=LOG_DIR,
        histogram_freq=0, 
        write_graph=True, 
        # profile_batch=(20,50) #run profiler on steps 20-40 - enable this line if you want to run profiler from the utils/ notebook
    )

### Training using tensorboard callback

While profiling does not work for managed Tensorboard at this time, you can inspect the profiler with an [inline Tensorboard in another notebook](https://www.tensorflow.org/tensorboard/tensorboard_in_notebooks). You may be prompted to install the tensorflow profiler library

In [ ]:
NUM_EPOCHS = 70
RUN_NAME = f'run-{EXPERIMENT_NAME}-{time.strftime("%Y%m%d-%H%M%S")}'#be sure to think about run and experiment naming strategies so names don't collide

#start the run to collect metrics - note `.log_parameters()` is available but not used

#start the timer and training
start_time = time.time()
layer_history = model.fit(
    train_dataset.unbatch().batch(batch_size),
    validation_data=valid_dataset,
    validation_freq=3,
    epochs=NUM_EPOCHS,
    # steps_per_epoch=2, #use this for development to run just a few steps
    validation_steps = 100,
    callbacks=[tensorboard_callback,
               UploadTBLogsBatchEnd()], #the tensorboard will be automatically associated with the experiment and log subsequent runs with this callback
    verbose=1
)

end_time = time.time()
val_keys = [v for v in layer_history.history.keys()]
runtime_mins = int((end_time - start_time) / 60)


vertex_ai.start_run(RUN_NAME, tensorboard=TB_RESOURCE_NAME)

vertex_ai.log_params({"layers": str(layer_sizes), 
                      "learning_rate": LR,
                        "num_epochs": epochs,
                        "batch_size": batch_size,
                     })

#gather the metrics for the last epoch to be saved in metrics
metrics_dict = {"train-time-minutes": runtime_mins}
_ = [metrics_dict.update({key: layer_history.history[key][-1]}) for key in val_keys]
vertex_ai.log_metrics(metrics_dict)
vertex_ai.end_run()

Epoch 1/70


2022-11-09 00:34:46.515483: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2022-11-09 00:34:47.244207: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8200


   3707/Unknown - 4882s 1s/step - batch_categorical_accuracy_at_1: 0.0000e+00 - batch_categorical_accuracy_at_5: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - loss: 50909.8305 - regularization_loss: 0.0000e+00 - total_loss: 50909.8305

2022-11-09 01:55:59.926501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 01:56:00.062321: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-09 01:56:00.103058: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-09 01:56:00.841399: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

View your Tensorboard at https://us-central1.tensorboard.googleusercontent.com/experiment/projects+934903580331+locations+us-central1+tensorboards+7336372589079560192+experiments+spotify-singe-node-train-full-data-v7-01
[2022-11-09T01:56:04] Started scanning logdir.
[2022-11-09T01:56:07] Total uploaded: 8 scalars, 0 tensors, 12 binary objects (6.1 MB)
3708/3708 [==============================] - 4893s 1s/step - batch_categorical_accuracy_at_1: 0.0000e+00 - batch_categorical_accuracy_at_5: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - loss: 50899.2761 - regularization_loss: 0.0000e+00 - total_loss: 50899.2761
Epoch 2/70
3707/3708 [============================>.] - ETA: 1s - batch_categorical_accuracy_at_1: 0.0000e+00 - batch_categorical_accuracy_at_5: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_cat

...

View your Tensorboard at https://us-central1.tensorboard.googleusercontent.com/experiment/projects+934903580331+locations+us-central1+tensorboards+7336372589079560192+experiments+spotify-singe-node-train-full-data-v7-01
[2022-11-09T16:26:25] Started scanning logdir.
[2022-11-09T16:26:35] Total uploaded: 72 scalars, 0 tensors, 12 binary objects (6.1 MB)
3708/3708 [==============================] - 4751s 1s/step - batch_categorical_accuracy_at_1: 0.0000e+00 - batch_categorical_accuracy_at_5: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - loss: 7756.7933 - regularization_loss: 0.0000e+00 - total_loss: 7756.7933
Epoch 8/70
3707/3708 [============================>.] - ETA: 1s - batch_categorical_accuracy_at_1: 0.0000e+00 - batch_categorical_accuracy_at_5: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_cate

2022-11-09 17:45:56.065276: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 17:45:56.208794: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-09 17:45:56.251314: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-09 17:45:56.992739: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

View your Tensorboard at https://us-central1.tensorboard.googleusercontent.com/experiment/projects+934903580331+locations+us-central1+tensorboards+7336372589079560192+experiments+spotify-singe-node-train-full-data-v7-01
[2022-11-09T17:45:58] Started scanning logdir.
[2022-11-09T17:46:08] Total uploaded: 80 scalars, 0 tensors, 12 binary objects (6.1 MB)
3708/3708 [==============================] - 4773s 1s/step - batch_categorical_accuracy_at_1: 0.0000e+00 - batch_categorical_accuracy_at_5: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - loss: 7736.9064 - regularization_loss: 0.0000e+00 - total_loss: 7736.9064
Epoch 9/70
3707/3708 [============================>.] - ETA: 1s - batch_categorical_accuracy_at_1: 0.0000e+00 - batch_categorical_accuracy_at_5: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_cate

2022-11-09 19:05:47.547252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-09 19:05:47.686335: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-09 19:05:47.730003: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-09 19:05:48.480374: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

View your Tensorboard at https://us-central1.tensorboard.googleusercontent.com/experiment/projects+934903580331+locations+us-central1+tensorboards+7336372589079560192+experiments+spotify-singe-node-train-full-data-v7-01
[2022-11-09T19:05:49] Started scanning logdir.
3608/3708 [============================>.] - ETA: 2:08 - batch_categorical_accuracy_at_1: 0.0000e+00 - batch_categorical_accuracy_at_5: 0.0000e+00 - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - loss: 7716.3438 - regularization_loss: 0.0000e+00 - total_loss: 7716.3438

#### You can access the experiment from the console via the experiment name you just declared:

![](img/experiment-console.png)

![](img/tensorboard.png)

### Also, while this is running - check out the Tensorboard profiler in `utils`.

![](img/tb-profiler.png)

### Run `nvtop` - check out the installation script in `utils` - `install_nvtop.sh`

![](img/nvtop-optimized.png)

In [ ]:
print(f"Total runtime: {runtime_mins} minutes")

### When complete you get a decent model with around 30-40 hit rate for top 1

![](img/tb-metrics.png)
![](img/tb-loss.png)

In [ ]:
#get metrics for the Vertex Experiment
metrics_dict

### Now, save the model

In [ ]:
# first, create the bucket to store the tensorflow models
# ! gsutil mb -l us-central1 $path

In [ ]:
#save the models

tf.saved_model.save(model.query_tower, export_dir=path + "/query_model")
tf.saved_model.save(model.candidate_tower, export_dir=path + "/candidate_model")

## Save the candidate embeddings to GCS for use in Matching Engine later
These will be the files we use for the index

This does the following
1) Create a tf pipeline to convert embeddings to numpy
2) Serialize the candidate song emgeddings with the song_uri index and save to gcs

In [ ]:
# create a tf function to convert any bad null values
def tf_if_null_return_zero(val):
    """
    this function fills in nans to zeros - sometimes happens in embedding calcs.
    this will clean the embedding inputs downstream
    """
    return(tf.clip_by_value(val, -1e12, 1e12)) # a trick to remove NANs post tf2.0

In [ ]:
candidate_embeddings = tt.parsed_candidate_dataset.batch(10000).map(lambda x: [x['track_uri_can'], tf_if_null_return_zero(model.candidate_tower(x))])

In [ ]:
# Save to the required format
# make sure you start out with a clean empty file for the append write
!rm candidate_embeddings.json > /dev/null 
!touch candidate_embeddings.json
for batch in candidate_embeddings:
    songs, embeddings = batch
    with open("candidate_embeddings.json", 'a') as f:
        for song, emb in zip(songs.numpy(), embeddings.numpy()):
            f.write('{"id":"' + str(song) + '","embedding":[' + ",".join(str(x) for x in list(emb)) + ']}')
            f.write("\n")

In [ ]:
tt.upload_blob('two-tower-models', 'candidate_embeddings.json', 'candidates/candidate_embeddings.json')

Do a quick line count from terminal - should look like this:

```
(base) jupyter@tf28-jsw-sep-a100:~/spotify_mpd_two_tower$ wc -l candidate_embeddings.json 
2249561 candidate_embeddings.json
```

### Finished

Go on to the [03 notebook](03-matching-engine.ipynb)

You should see results similar to the screenshot below
![](img/embeddings.png)